# 3️⃣ PTL 변환: 학습된 모델 모바일 변환

이 노트북은 2_train.ipynb에서 학습한 3가지 KD 모델을 PTL 파일로 변환합니다.

**변환 대상:**
- Output KD (`weights/finetuned_output.pth`)
- Feature KD (`weights/finetuned_feature.pth`)
- FaKD (`weights/finetuned_fakd.pth`)

**입력 해상도:** 320, 640, 1280, 1920 (4가지)

**총 생성 파일:** 3 모델 × 4 해상도 = 12개 PTL 파일

**주의:** 메모리 사용량이 크므로 순차적으로 한 개씩 변환합니다.

---
## ⚙️ 0. 경로 설정 (이 셀만 수정하세요!)

In [ ]:
# ============================================================
# 🔧 사용자 설정 - 1_setup.ipynb와 동일하게!
# ============================================================
BASE_DIR = "my_ssl"
VENV_NAME = "ssl_env"
PYTHON_VERSION = "3.12"
REPO_NAME = "uos-ssl-project"

# ============================================================
# 🔒 아래는 자동 설정 (수정 불필요)
# ============================================================
import os
import sys
import gc
import subprocess
os.environ['MPLBACKEND'] = 'Agg'

home_dir = os.path.expanduser('~')
base_path = os.path.join(home_dir, BASE_DIR)
venv_path = os.path.join(base_path, VENV_NAME)
site_packages = os.path.join(venv_path, 'lib', f'python{PYTHON_VERSION}', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')
project_dir = os.path.join(base_path, REPO_NAME)
ai_dir = os.path.join(project_dir, 'ai')

if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

%cd {ai_dir}

import torch

# 경로 검증
print("="*60)
print("🔍 경로 검증")
print("="*60)
paths_ok = True
for name, path in [("AI Dir", ai_dir), ("Python", python_exec), ("Venv", venv_path)]:
    if os.path.exists(path):
        print(f"✅ {name}: {path}")
    else:
        print(f"❌ {name}: {path} (없음!)")
        paths_ok = False

if not paths_ok:
    raise RuntimeError("경로 설정 오류! 위의 사용자 설정을 확인하세요.")

print(f"\nPyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA:    {torch.cuda.get_device_name(0)}")
print("="*60)

In [ ]:
# ============================================================
# 유틸리티 함수
# ============================================================
def convert_to_ptl(weights_path, input_size, output_path):
    """PTL 변환 실행 후 에러 체크"""
    cmd = [
        python_exec, 'scripts/convert_to_ptl.py',
        '--weights', weights_path,
        '--input_size', str(input_size),
        '--upscale', '2',
        '--output', output_path
    ]
    
    result = subprocess.run(cmd, cwd=ai_dir)
    
    # 메모리 정리
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    if result.returncode != 0:
        print(f"❌ 변환 실패! (exit code: {result.returncode})")
        return False
    
    if os.path.exists(output_path):
        size_mb = os.path.getsize(output_path) / (1024 * 1024)
        print(f"✅ 완료: {output_path} ({size_mb:.1f} MB)")
        return True
    else:
        print(f"❌ 파일 생성 실패: {output_path}")
        return False

## 3.1 변환 대상 모델 확인

In [ ]:
print("📁 변환 대상 모델 확인")
print("="*60)

models = [
    ('output', 'weights/finetuned_output.pth', 'Output KD'),
    ('feature', 'weights/finetuned_feature.pth', 'Feature KD'),
    ('fakd', 'weights/finetuned_fakd.pth', 'FaKD'),
]

available_models = []
for model_id, path, name in models:
    full_path = os.path.join(ai_dir, path)
    if os.path.exists(full_path):
        size = os.path.getsize(full_path) / (1024 * 1024)
        print(f"✅ {name:12} | {path} ({size:.1f} MB)")
        available_models.append((model_id, path, name))
    else:
        print(f"❌ {name:12} | {path} (없음)")

print(f"\n📊 변환 가능한 모델: {len(available_models)}/3")

if len(available_models) == 0:
    print("\n⚠️ 변환할 모델이 없습니다!")
    print("   먼저 2_train.ipynb를 실행하여 모델을 학습하세요.")

---
## 3.2 Output KD 모델 PTL 변환

In [ ]:
print("="*60)
print("🔄 Output KD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_output.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(os.path.join(ai_dir, weights_path)):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
    print("   먼저 2_train.ipynb에서 Output KD 학습을 완료하세요.")
else:
    success_count = 0
    fail_count = 0
    
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] Output KD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/output_kd_{size}.ptl"
        full_output = os.path.join(ai_dir, output_path)
        
        if os.path.exists(full_output):
            size_mb = os.path.getsize(full_output) / (1024 * 1024)
            print(f"✅ 이미 존재: {output_path} ({size_mb:.1f} MB)")
            success_count += 1
            continue
        
        if convert_to_ptl(weights_path, size, output_path):
            success_count += 1
        else:
            fail_count += 1
    
    print("\n" + "="*60)
    if fail_count == 0:
        print(f"✅ Output KD PTL 변환 완료! ({success_count}/4 성공)")
    else:
        print(f"⚠️ Output KD PTL 변환 일부 실패! ({success_count}/4 성공, {fail_count}/4 실패)")

---
## 3.3 Feature KD 모델 PTL 변환

In [ ]:
print("="*60)
print("🔄 Feature KD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_feature.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(os.path.join(ai_dir, weights_path)):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
    print("   먼저 2_train.ipynb에서 Feature KD 학습을 완료하세요.")
else:
    success_count = 0
    fail_count = 0
    
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] Feature KD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/feature_kd_{size}.ptl"
        full_output = os.path.join(ai_dir, output_path)
        
        if os.path.exists(full_output):
            size_mb = os.path.getsize(full_output) / (1024 * 1024)
            print(f"✅ 이미 존재: {output_path} ({size_mb:.1f} MB)")
            success_count += 1
            continue
        
        if convert_to_ptl(weights_path, size, output_path):
            success_count += 1
        else:
            fail_count += 1
    
    print("\n" + "="*60)
    if fail_count == 0:
        print(f"✅ Feature KD PTL 변환 완료! ({success_count}/4 성공)")
    else:
        print(f"⚠️ Feature KD PTL 변환 일부 실패! ({success_count}/4 성공, {fail_count}/4 실패)")

---
## 3.4 FaKD 모델 PTL 변환

In [ ]:
print("="*60)
print("🔄 FaKD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_fakd.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(os.path.join(ai_dir, weights_path)):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
    print("   먼저 2_train.ipynb에서 FaKD 학습을 완료하세요.")
else:
    success_count = 0
    fail_count = 0
    
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] FaKD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/fakd_kd_{size}.ptl"
        full_output = os.path.join(ai_dir, output_path)
        
        if os.path.exists(full_output):
            size_mb = os.path.getsize(full_output) / (1024 * 1024)
            print(f"✅ 이미 존재: {output_path} ({size_mb:.1f} MB)")
            success_count += 1
            continue
        
        if convert_to_ptl(weights_path, size, output_path):
            success_count += 1
        else:
            fail_count += 1
    
    print("\n" + "="*60)
    if fail_count == 0:
        print(f"✅ FaKD PTL 변환 완료! ({success_count}/4 성공)")
    else:
        print(f"⚠️ FaKD PTL 변환 일부 실패! ({success_count}/4 성공, {fail_count}/4 실패)")

---
## 3.5 생성된 PTL 파일 확인

In [ ]:
print("="*60)
print("📁 생성된 PTL 파일 목록")
print("="*60)

weights_dir = os.path.join(ai_dir, 'weights')

def list_ptl_files(pattern, label):
    """특정 패턴의 PTL 파일 목록 출력"""
    print(f"\n🔹 {label}:")
    if os.path.exists(weights_dir):
        files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.ptl') and pattern in f])
        if files:
            for f in files:
                path = os.path.join(weights_dir, f)
                size_mb = os.path.getsize(path) / (1024 * 1024)
                print(f"   {f} ({size_mb:.1f} MB)")
            return len(files)
    print("   없음")
    return 0

total = 0
total += list_ptl_files('CATANet-L_x2', '원본 CATANet-L x2')
total += list_ptl_files('output_kd', 'Output KD')
total += list_ptl_files('feature_kd', 'Feature KD')
total += list_ptl_files('fakd_kd', 'FaKD')

print("\n" + "="*60)
print(f"📊 총 PTL 파일: {total}개")
print("="*60)
print("\n💡 다음: 4_benchmark.ipynb에서 성능 평가")